In [1]:
!pip install git+https://github.com/canesche/nvcc4jupyter.git
!git clone https://github.com/canesche/nvcc4jupyter
%load_ext nvcc_plugin

  Cloning https://github.com/canesche/nvcc4jupyter.git to /tmp/pip-req-build-vwjxhur3
  Running command git clone -q https://github.com/canesche/nvcc4jupyter.git /tmp/pip-req-build-vwjxhur3
  Created wheel for ColabPlugin: filename=ColabPlugin-blind-py3-none-any.whl size=12727 sha256=28e3d5d526d4f33c9f80836abb2fea6fc4bbcb9808270f161e33d21f33b9e3b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-yy_052cz/wheels/06/76/5f/88825d3256ab9fe9e4386e23ad33a2c41a2e4dc94f0addff44
Failed to build ColabPlugin
    Running setup.py install for ColabPlugin ... done
  DEPRECATION: ColabPlugin was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
Cloning into 'nvcc4jupyter'...
remote: Enumerating objects: 1147, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (271/2


Considere que desejamos gerar um array B através de um array A de mesmo tamanho, de modo que, cada elemento `B[i]` é dado pela soma do elemento `A[i]` mais os `r` elementos anteriores à posição `i` em A mais os `r` elementos posteriores, ou seja:
```
for(j=i-r; j<=i+r; j++)
    B[i]+=A[j]
```
O código dado a seguir já faz o cálculo do array B (armazenado em `bHost`) de forma sequencial. Escreva um kernel CUDA, bem como todo o restante do código necessário para executá-lo, de modo a permitir o cálculo de B de forma paralela.

As posições inválidas no início e final do array A devem ser simplesmente descartadas durante a soma, ou seja, para cálculo de `B[10]` com `r=32`, o resultado será a soma dos elementos de `A[0]` até `A[42]`.

Se atente para os seguinte requisitos:
* O cálculo dos elementos de B deve ser realizado de forma paralela;
* A memória compartilhada da GPU deve ser utilizada de forma a deixar a operação mais eficiente;
* O código deve tratar corretamente arrays grandes divididos em mais de um bloco.

Dicas:
* O código abaixo utiliza a constante `RANGE` para definir o valor de `r` e a constante `BLOCKSIZE` para definir o tamanho de cada bloco do grid;
* Por simplicidade, você pode considerar que o tamanho do array A será sempre múltiplo de `BLOCKSIZE`;
* Considere também que o tamanho de `r` será no máximo igual à metade de `BLOCKSIZE`.
* A estratégia utilizada pela versão sequencial de testar cada posição (para descobrir se é válida) não é ótima. Muitas comparações desnecessárias são realizadas ao calcular as posições intermediárias. Na versão paralela, é melhor preecher as posições inválidas com o valor 0 (zero) ao copiar os dados para a memória compartilhada.

In [2]:
%%gpu
#include<stdio.h>
#include<cuda.h>

#define BLOCKSIZE 1024   //threads por bloco
#define RANGE 256       //deslocamento utilizado para cálculo de B

void initArray(int *a, int n) {
    for (int i =0; i<n; ++i) 
        a[i]=i;
}

void printArray(int *a, int n) {
    int maxPrint=20;
    for (int i =0; i<(n>maxPrint ? maxPrint : n); ++i) 
        printf("%3d ",a[i]);
    if (n>maxPrint) printf("... (array truncado)");
    printf("\n");
}

bool checkArray(int *a, int *b, int n) {
    for (int i =0; i<n; ++i) 
        if (a[i]!=b[i])
            return false;
    return true;
}

void execHost(int *A, int n, int *B) {
    for(int i=0; i<n; i++) {
        B[i]=0;
        for(int j=i-RANGE; j<=i+RANGE; j++)  
            if(j >= 0 && j<n)
                B[i]+=A[j];
    }
}

__global__ void execDevice(int *A, int n, int *B){
      const int N = RANGE*2 + BLOCKSIZE; //a memoria auxiliar deve ter o tamanho do bloco
                                         // mais os espaços para os dados do RANGE*2(atras e na frente)
      __shared__ int s_A[N];

      int tId = blockIdx.x *blockDim.x + threadIdx.x; //id da thread no vetor a
      int tx = threadIdx.x; //id da thread no bloco

      s_A[tx+RANGE] = A[tId];   //posições que não são extremos podem ser carregadas direto na memoria

      if(tx == 0){  //se é a primeira thread do bloco as posições a esquerda devem ser preenchidas
          for(int i = 0;i<RANGE+1;i++){ 
              if(tId-i < 0)    //se não é uma posição válida, colocar 0 na memória
                s_A[tx-i + RANGE] = 0;
              else s_A[tx-i + RANGE] = A[tId-i];
          }
      }
      if(tx == BLOCKSIZE -1){ // se for a ultima thread do bloco as posições a direita devem ser preenchidas com 0
          for(int j = 0;j< RANGE+1;j++){
              if(tId+j >= n)       //se não é uma posição válida, colocar 0 na memória
                  s_A[tx+j + RANGE] = 0;
              else s_A[tx+j + RANGE] = A[tId + j];
          }
      }

      __syncthreads();


      int soma = 0; //variavel auxiliar, evita atualizar direto na memoria global

      for(int i = tx;i<= tx+2*RANGE;i++){
          soma += s_A[i];
      }
      
      B[tId] = soma;

}


int main() {
    int size=1<<20;             //tamanho dos arrays A e B
    int dsize=size*sizeof(int); //tamanho dos dados
    int *a;
    int *b;                     //array B a ser calculado na GPU
    int *bHost;                 //array B calculado na CPU
    
    int *d_A;
    int *d_B;

    a=(int*)malloc(dsize);
    b=(int*)malloc(dsize);
    bHost=(int*)malloc(dsize);

    // Aloca os vetores que serão usados no calc
    cudaMalloc((void **) &d_A, dsize);
    cudaMalloc((void **) &d_B, dsize);

    initArray(a,size);
    printf("    A: ");
    printArray(a,size);

    // Faz uma cópia de A que será passado para a função
    cudaMemcpy(d_A, a, dsize, cudaMemcpyHostToDevice);

    dim3 grid (ceil(size*1.0/BLOCKSIZE), 1, 1);
    dim3 block (BLOCKSIZE, 1, 1);

    printf("B-GPU: ");
    execDevice <<<grid, block>>>(d_A, size, d_B);

    // copia de volta para um B
    cudaMemcpy(b, d_B, dsize, cudaMemcpyDeviceToHost);
    printArray(b,size);

    printf("B-CPU: ");
    execHost(a, size, bHost);
    printArray(bHost,size);

    if (checkArray(b,bHost,size))
        printf("Resultados iguais\n");
    else
        printf("Resultados diferentes\n");

    free(bHost);
    free(a);
    free(b);
    cudaFree(d_A);
    cudaFree(d_B);
    return 0;
}

    A:   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 ... (array truncado)
B-GPU: 32896 33153 33411 33670 33930 34191 34453 34716 34980 35245 35511 35778 36046 36315 36585 36856 37128 37401 37675 37950 ... (array truncado)
B-CPU: 32896 33153 33411 33670 33930 34191 34453 34716 34980 35245 35511 35778 36046 36315 36585 36856 37128 37401 37675 37950 ... (array truncado)
Resultados iguais

